In [ ]:
!pip uninstall -y torch
!pip install torch==1.8.2+cpu -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install -q cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
import torch_xla


Found existing installation: torch 1.8.2+cpu
Uninstalling torch-1.8.2+cpu:
  Successfully uninstalled torch-1.8.2+cpu
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
  Using cached https://download.pytorch.org/whl/lts/1.8/cpu/torch-1.8.2%2Bcpu-cp37-cp37m-linux_x86_64.whl (169.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.2+cpu which is incompatible.


In [ ]:
import torch_xla


In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import nltk
import re
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
books = sorted(glob.glob('/content/Book*'))
books

["/content/Book 1 - The Philosopher's Stone.txt",
 '/content/Book 2 - The Chamber of Secrets.txt',
 '/content/Book 3 - The Prisoner of Azkaban.txt',
 '/content/Book 4 - The Goblet of Fire.txt',
 '/content/Book 5 - The Order of the Phoenix.txt',
 '/content/Book 6 - The Half Blood Prince.txt',
 '/content/Book 7 - The Deathly Hallows.txt']

In [ ]:
def clean_text(text):
  pattern = '(page|Page|PAGE)(\s+\|\s+)([0-9]+)(.*)$'
  cleaned = re.sub('\s$','',text, flags=re.MULTILINE)
  p = re.compile(pattern, re.MULTILINE)
  cleaned = p.sub(" ", cleaned)
  return cleaned

In [ ]:
text=""
for book in books:
  with open(book) as f:
    print('Reading', book.split('/')[-1])
    temp = f.read()
    text = text + clean_text(temp)


Reading Book 1 - The Philosopher's Stone.txt
Reading Book 2 - The Chamber of Secrets.txt
Reading Book 3 - The Prisoner of Azkaban.txt
Reading Book 4 - The Goblet of Fire.txt
Reading Book 5 - The Order of the Phoenix.txt
Reading Book 6 - The Half Blood Prince.txt
Reading Book 7 - The Deathly Hallows.txt


In [ ]:

with open("/content/Cleaned.txt",'w') as f:
     f.writelines(text)

In [ ]:
text = ""
with open("/content/drive/MyDrive/Harry Potter/Cleaned.txt",'r') as f:
  text = f.readlines()

In [ ]:
text[0:20]

['/\n',
 'THE BOY WHO LIVED\n',
 'Mr. and Mrs. Dursley, of number four, Privet Drive,\n',
 'were proud to say that they were perfectly normal,\n',
 'thank you very much. They were the last people you’d\n',
 'expect to be involved in anything strange or\n',
 'mysterious, because they just didn’t hold with such\n',
 'nonsense.\n',
 'Mr. Dursley was the director of a firm called\n',
 'Grunnings, which made drills. He was a big, beefy\n',
 'man with hardly any neck, although he did have a\n',
 'very large mustache. Mrs. Dursley was thin and\n',
 'blonde and had nearly twice the usual amount of\n',
 'neck, which came in very useful as she spent so\n',
 'much of her time craning over garden fences, spying\n',
 'on the neighbors. The Dursley s had a small son\n',
 'called Dudley and in their opinion there was no finer\n',
 'boy anywhere.\n',
 'The Dursleys had everything they wanted, but they\n',
 'also had a secret, and their greatest fear was that\n']

In [ ]:
train_harry = text[1:round(0.9*len(text))]
val_harry = text[round(0.9*len(text)):]

In [ ]:
with open("/content/drive/MyDrive/Harry Potter/train_harry.txt",'w') as f:
     f.writelines(train_harry)
with open("/content/drive/MyDrive/Harry Potter/val_harry.txt",'w') as f:
     f.writelines(val_harry)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

In [ ]:
paths = ['/content/drive/MyDrive/Harry Potter/Cleaned.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
tokenizer.save_model("/content/drive/MyDrive/Harry Potter/")

['/content/drive/MyDrive/Harry Potter/vocab.json',
 '/content/drive/MyDrive/Harry Potter/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "/content/drive/MyDrive/Harry Potter/vocab.json",
    "/content/drive/MyDrive/Harry Potter/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
%%time
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Harry Potter/", max_len=512)

Didn't find file /content/drive/MyDrive/Harry Potter/added_tokens.json. We won't load it.
Didn't find file /content/drive/MyDrive/Harry Potter/special_tokens_map.json. We won't load it.
Didn't find file /content/drive/MyDrive/Harry Potter/tokenizer_config.json. We won't load it.
Didn't find file /content/drive/MyDrive/Harry Potter/tokenizer.json. We won't load it.
loading file /content/drive/MyDrive/Harry Potter/vocab.json
loading file /content/drive/MyDrive/Harry Potter/merges.txt
loading file None
loading file None
loading file None
loading file None
file /content/drive/MyDrive/Harry Potter/config.json not found


CPU times: user 75 ms, sys: 18 ms, total: 93 ms
Wall time: 133 ms


In [ ]:
from tokenizers.decoders import ByteLevel

tokenizer.encode("Harry stood up and said")


[417, 1140, 415, 298, 324]

In [ ]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/drive/MyDrive/Harry Potter/train_harry.txt',
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/drive/MyDrive/Harry Potter/val_harry.txt',
    block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file /content/drive/MyDrive/Harry Potter/cached_lm_GPT2Tokenizer_128_train_harry.txt [took 0.119 s]
Loading features from cached file /content/drive/MyDrive/Harry Potter/cached_lm_GPT2Tokenizer_128_val_harry.txt [took 0.013 s]


In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Harry Potter/Model/')
device = xm.xla_device()
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

loading configuration file /content/drive/MyDrive/Harry Potter/Model/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/Harry Potter/Model/",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.10.2",
  "use_cache": true,
  "vocab_size": 502

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/Harry Potter/Model', # the output directory for the model predictions and checkpoints
    overwrite_output_dir = True, # overwrite the content of the output directory
    per_device_train_batch_size = 64, # the batch size for training
    per_device_eval_batch_size = 64, # the batch size for evaluation
    learning_rate = 8e-5, # defaults to 5e-5
    num_train_epochs = 10,
    save_steps = 300,
    logging_first_step = True,
    logging_steps = 100 # total number of training epochs to perform
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 11026
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1730


Step,Training Loss
1,0.046100
100,0.075800
200,0.086300
300,0.082200
400,0.079300
500,0.078000
600,0.072800
700,0.072300
800,0.067100
900,0.065000


Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-300
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-300/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-600
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-600/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-900
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-900/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-900/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-1200
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-1200/config.json


TrainOutput(global_step=1730, training_loss=0.06576535908042351, metrics={'train_runtime': 974.5024, 'train_samples_per_second': 113.145, 'train_steps_per_second': 1.775, 'total_flos': 7202514862080000.0, 'train_loss': 0.06576535908042351, 'epoch': 10.0})

In [ ]:
  trainer.save_model('/content/drive/MyDrive/Harry Potter/Model/')


Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/pytorch_model.bin


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 11026
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1730


Step,Training Loss
1,0.040500
100,0.051300
200,0.051200
300,0.049900
400,0.049300
500,0.049700
600,0.048400
700,0.048300
800,0.047700
900,0.047300


Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-300
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-300/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-600
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-600/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-600/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-900
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-900/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-900/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/checkpoint-1200
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/checkpoint-1200/config.json


TrainOutput(global_step=1730, training_loss=0.0476233291380495, metrics={'train_runtime': 985.8978, 'train_samples_per_second': 111.837, 'train_steps_per_second': 1.755, 'total_flos': 7202514862080000.0, 'train_loss': 0.0476233291380495, 'epoch': 10.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/Harry Potter/Model/')


Saving model checkpoint to /content/drive/MyDrive/Harry Potter/Model/
Configuration saved in /content/drive/MyDrive/Harry Potter/Model/config.json
Model weights saved in /content/drive/MyDrive/Harry Potter/Model/pytorch_model.bin


In [ ]:
%cp "/content/drive/MyDrive/Harry Potter/Model/checkpoint-2400/pytorch_model.bin" "/content/drive/MyDrive/Harry Potter/Model/temp/"

In [ ]:
%cp "/content/drive/MyDrive/Harry Potter/Model/checkpoint-2400/optimizer.pt" "/content/drive/MyDrive/Harry Potter/Model/temp/"

In [ ]:
generator = pipeline('text-generation', tokenizer=tokenizer , model='/content/drive/MyDrive/Harry Potter/Model/')


loading configuration file /content/drive/MyDrive/Harry Potter/Model/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/Harry Potter/Model/",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.10.2",
  "use_cache": true,
  "vocab_size": 502

In [ ]:
print(generator('Harry picked up the wand and', max_length=300)[0]['generated_text'])

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Harry picked up the wand and cast it onto his

hands. “Here — here — HERE — ” Harry pointed it at the

stool, as if there would be no effect to anything else,

but all the spell hit the shelf, causing Professor

McGonagall to fumble in her seat with one hand.

Several feet away, however, Ron was pointing it to

Harry.

“Sirius!”

“Harry!”

Harry and Hermione scrambled around him,

looking around.

“You knew — Sirius Black, are ’you? He’s supposed to

be safe.”

“It’s my name,” said Snape, his eyes glittering. “What is it?”

He heaved the wand into his lap. His hand flew to meet Harry’s.

Something odd about Sirius looked like a quill.

“I’m a quill.”

Harry shook it. His face looked up. It looked very white.

“Good idea, Snape, Potter,’s hand was shaking.

It looked very cold. Had it gone. They shook it. His eyes fell forward. But Snape straightened his eyes

shouldered Harry’s wand, straightened Harry’s


In [ ]:
print(generator('Ayush and Deep were walking towards Suprasiddha', max_length=200)[0]['generated_text'])

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ayush and Deep were walking towards Suprasiddhaa
Beauxbatons dryly, which exploded at the thought that
the house-elf drifted up to the top of the stairs; and
her pimples were visible, they were quite keen to let
them inside the staff members of the D.A. closest
very distracting.
“You could just leave her alone if she saw you,”
said Ron, leaning back to his seat and holding
his broomstick. “You went to sit in the Great Hall,
don’t you?”
“Yeah, I suppose,” said Harry, who could feel cold sweat
on his face.
“But he didn’t do it at all right. “I bet I didn’t expect you anything.”
“ relief. She’d probably knew something really used to turn up to tell her to turn up to
her at the whole.”
too. She left me because she was probably told me because she


In [ ]:
!rm '/content/drive/MyDrive/Harry Potter/Model/pytorch_model.bin'